### Instalação das dependências
Como este notebook está dentro do projeto do TCC é necessário instalar somente o hyperas. Caso não tenha as outras dependências basta rodar o arquivo de **requirements.txt** presente na raiz da aplicação.

In [1]:
!pip install hyperas

You should consider upgrading via the 'd:\documentos\harã stuffs\faculdade\tcc\codes\tcc-rede-neural-siamesa\venv\scripts\python.exe -m pip install --upgrade pip' command.


### Importações das dependências

In [2]:
import os
import itertools
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Embedding, LSTM, Lambda, Conv1D, Dense, Dropout, Activation, Bidirectional
from tensorflow.python.keras import backend as k
from tensorflow.python.keras.layers import Lambda, Reshape, dot
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe

### Definição dos paths padrões

In [3]:
DATA_FILES_INDEX_VECTORS_PATH = os.path.join(os.path.dirname(os.path.abspath("")), "data", "processed/index_vectors")
DATA_FILES_EMBEDDING_MATRICES_PATH = os.path.join(os.path.dirname(os.path.abspath("")), "data", "processed/embedding_matrices")

### Carregando os vetores de índices
São os vetores que são alimentados na entrada da rede neural siamesa, onde cada valor representa **o índice correspondente na matriz de incoporação**. Ambos (matriz de incoporação e vetor de índice) são armazenados na aplicação após ser aplicado o processo de estruturação de dados. Para cada word embedding utilizado foi criado um arquivo que armazena os vetores de índices para os datasets: cru (raw), sem stopwords (sw) e sem stopwords e com lematização (sw_lemmatization).

In [4]:
def data():
    ### FUNÇÕES INTERNAS ###
    
    def load_index_vector_dataframe(filename):
        dataframe = pd.read_csv(filename)

        for q in ['phrase1', 'phrase2']:
            dataframe[q + '_n'] = dataframe[q].apply(lambda x: [int(i) for i in x.replace('[', '').replace(']', '').split(',')])

        return dataframe

    def split_data_train(train_dataframe):
        x_phrases = train_dataframe[['phrase1_n', 'phrase2_n']]

        train_dataframe.label = pd.Categorical(train_dataframe.label)
        train_dataframe['label'] = train_dataframe.label.cat.codes
        y_labels = train_dataframe['label']

        return x_phrases, y_labels
    
    def split_and_zero_padding(dataframe, max_seq_length):
        # Split to dicts
        side_phrases = {'left': dataframe['phrase1_n'], 'right': dataframe['phrase2_n']}
        dataset = None

        # Zero padding
        for dataset, side in itertools.product([side_phrases], ['left', 'right']):
            dataset[side] = pad_sequences(dataset[side], padding='pre', truncating='post', maxlen=max_seq_length)

        return dataset
    
    ### FUNÇÕES INTERNAS ###
    
    
    ### ARQUIVOS VETOR INDICES ###

    DATA_FILES_INDEX_VECTORS_PATH = os.path.join(os.path.dirname(os.path.abspath("")), "data", "processed/index_vectors")
    
    # Word2vec Google News
    raw_w2v_GN = os.path.join(DATA_FILES_INDEX_VECTORS_PATH, "training-raw-w2v_GN.csv")
    sw_w2v_GN = os.path.join(DATA_FILES_INDEX_VECTORS_PATH, "training-sw-w2v_GN.csv")
    sw_lemma_w2v_GN = os.path.join(DATA_FILES_INDEX_VECTORS_PATH, "training-sw-lemmatization-w2v_GN.csv")
    
    # Word2vec Wikipedia
    
    # Glove Wikipedia + Gigaword
    
    # Glove Common Crawl
    
    ### ARQUIVOS WORD EMBEDDING ###
    
    
    ### TIPOS DE MAX_SEQ_LENGTH ###

    MAX_SEQ_LENGTH_RAW = 17
    MAX_SEQ_LENGTH_SW = 9
    MAX_SEQ_LENGTH_SW_LEMMA = 9

    max_seq_length = MAX_SEQ_LENGTH_RAW

    ### TIPOS DE MAX_SEQ_LENGTH ###
    
    
    # Carregamento do vetor de índices através do pandas    
    train_dataframe = load_index_vector_dataframe(raw_w2v_GN)
    
    # Divisão do dataset de entrada da rede neural em TREINAMENTO e TESTE (PREDIÇÃO)
    x_phrases, y_labels = split_data_train(train_dataframe)
    
    # Divisão 90/10 de treinamento e teste
    x_train, x_test, y_train, y_test = train_test_split(x_phrases, y_labels, test_size=0.1, random_state=0, stratify=train_dataframe['label'])
    
    # Divisão dos dados de TREINAMENTO de entrada entre a parte esquerda e direita das subredes e zero padding à esquerda dos dados de TREINAMENTO (retorno em ndarray)
    x_train = split_and_zero_padding(x_train, max_seq_length)

    # Divisão dos dados de TESTE (predição) de entrada entre a parte esquerda e direita das subredes e zero padding à esquerda dos dados de TESTE (predição) (retorno em ndarray)
    x_test = split_and_zero_padding(x_test, max_seq_length)

    # Conversão para numpy das labels
    y_train = y_train.values
    y_test = y_test.values

    return x_train, y_train, x_test, y_test

### Criação do modelo da rede siamesa
Criação do modelo da rede siamesa utilizando como subredes internas a arquitetura *LSTM (Long Short Term Memory)*.

In [5]:
def create_model(x_train, y_train, x_test, y_test):
    
    ### FUNÇÕES INTERNAS ###

    # Distância de Manhattan
    def define_manhattan_model(shared_model, max_seq_length):
        def calculate_manhattan_distance(left_output, right_output):          
            def __exponent_neg_manhattan_distance(left, right):
                return k.exp(-k.sum(k.abs(left - right), axis=1, keepdims=True))

            manhattan_distance = Lambda(
                function=lambda x: __exponent_neg_manhattan_distance(x[0], x[1]),
                output_shape=lambda x: (x[0][0], 1)
            )([left_output, right_output])

            return manhattan_distance

        # The visible layer
        left_input = Input(shape=(max_seq_length,), dtype='int32')
        right_input = Input(shape=(max_seq_length,), dtype='int32')

        # Pack it all up into a Manhattan Distance model
        malstm_distance = calculate_manhattan_distance(shared_model(left_input), shared_model(right_input))
        manhattan_model = Model(inputs=[left_input, right_input], outputs=[malstm_distance])

        return manhattan_model

    # Similaridade de Cossenos
    def define_cosine_model(shared_model, max_seq_length):
        def calculate_cosine_distance(left_output, right_output):
            cos_distance = dot([left_output, right_output], axes=1, normalize=True)
            cos_distance = Reshape((1,))(cos_distance)
            cos_similarity = Lambda(lambda x: 1 - x)(cos_distance)

            return cos_similarity

        left_input = Input(shape=(max_seq_length,))
        right_input = Input(shape=(max_seq_length,))

        cosine_distance = calculate_cosine_distance(shared_model(left_input), shared_model(right_input))
        cosine_model = Model(inputs=[left_input, right_input], outputs=[cosine_distance])

        return cosine_model

    # Distância Euclidiana
    def define_euclidean_model(shared_model, max_seq_length):
        def calculate_euclidean_distance(vects):
            x, y = vects
            sum_square = k.sum(k.square(x - y), axis=1, keepdims=True)

            return k.sqrt(k.maximum(sum_square, k.epsilon()))

        def dist_output_shape(shapes):
            shape1, shape2 = shapes

            return (shape1[0], 1)

        left_input = Input(shape=(max_seq_length,))
        right_input = Input(shape=(max_seq_length,))

        euclidean_distance = Lambda(
            calculate_euclidean_distance,
            output_shape=dist_output_shape
        )([shared_model(left_input), shared_model(right_input)])
        euclidean_model = Model(inputs=[left_input, right_input], outputs=[euclidean_distance])

        return euclidean_model

    ### FUNÇÕES INTERNAS ###


    ### TIPOS DE MAX_SEQ_LENGTH ###

    MAX_SEQ_LENGTH_RAW = 17
    MAX_SEQ_LENGTH_SW = 9
    MAX_SEQ_LENGTH_SW_LEMMA = 9

    max_seq_length = MAX_SEQ_LENGTH_RAW

    ### TIPOS DE MAX_SEQ_LENGTH ###


    ### ARQUIVOS EMBEDDINGS ###
    
    DATA_FILES_EMBEDDING_MATRICES_PATH = os.path.join(os.path.dirname(os.path.abspath("")), "data", "processed/embedding_matrices")
    
    # Word2vec Google News
    raw_w2v_GN = os.path.join(DATA_FILES_EMBEDDING_MATRICES_PATH, "raw-w2v_GN.npy")
    sw_w2v_GN = os.path.join(DATA_FILES_EMBEDDING_MATRICES_PATH, "sw-w2v_GN.npy")
    sw_lemma_w2v_GN = os.path.join(DATA_FILES_EMBEDDING_MATRICES_PATH, "sw-lemmatization-w2v_GN.npy")

    # Word2vec Wikipedia

    # Glove Wikipedia + Gigaword

    # Glove Common Crawl

    ### ARQUIVOS EMBEDDINGS ###


    # Definição do modelo compartilhado (shared model) entre as subredes, pois são idênticas
    embedding_dim = 300
    embedding_matrix = np.load(raw_w2v_GN)

    shared_model = Sequential()
    shared_model.add(
        Embedding(
            len(embedding_matrix),
            embedding_dim,
            weights=[embedding_matrix],
            input_shape=(max_seq_length,),
            trainable=False
        )
    )
    
    shared_model.add(Dropout({{uniform(0, 1)}}))
    shared_model.add(Bidirectional(LSTM(
        {{choice([64, 128, 256, 512])}},
        kernel_initializer={{choice([tf.keras.initializers.VarianceScaling(scale=1.0, mode='fan_in', distribution='truncated_normal',seed=1), tf.keras.initializers.glorot_normal(seed=1)])}},
        activation='softsign',
        recurrent_activation='sigmoid',
        dropout=0.0,
        recurrent_dropout={{uniform(0, 1)}},
        implementation=1
    )))
    shared_model.add(Activation({{choice(['relu', 'selu', 'elu'])}}))
    shared_model.add(Dense(1, activation='sigmoid'))

    # Define o lado esquerdo e direito das subredes a partir da shared_model. Também define a medida/função de similaridade usada na camada de merge na saída da rede
    model = define_manhattan_model(shared_model, max_seq_length)

    # Compilação do modelo da rede siamesa
    model.compile(
        loss={{choice([tf.keras.losses.MeanSquaredError(), tfa.losses.ContrastiveLoss(), tfa.losses.TripletSemiHardLoss(), tfa.losses.TripletHardLoss(), tf.keras.losses.BinaryCrossentropy()])}},
        optimizer={{choice([tf.keras.optimizers.Adadelta(learning_rate=0.1, rho=0.95, epsilon=1e-07, name='Adadelta', clipnorm=1.5),
                            tf.keras.optimizers.Adamax(learning_rate=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"),
                            tf.keras.optimizers.Adagrad(learning_rate=0.1,  initial_accumulator_value=0.1,epsilon=1e-07, name='Adagrad', clipnorm=1.5),
                            tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.0, nesterov=False, name='SGD'),
                            tf.keras.optimizers.Adam(learning_rate=0.1),
                            tf.keras.optimizers.RMSprop(learning_rate=0.1)])}},
        metrics=['accuracy']
    )

    # Execução do treinamento da rede
    training_history = model.fit(
        [x_train['left'], x_train['right']],
        y_train,
        batch_size={{choice([16, 32, 64, 128, 256])}},
        epochs={{choice([10, 20])}},
        verbose=2,
        validation_split=0.2
    )
    
    return {'loss': -(np.amax(training_history.history['val_accuracy'])), 'status': STATUS_OK, 'model': model}

In [6]:
# Partes substituidas para teste
# {{choice([50, 100, 150, 200])}}
# optimizer= tf.keras.optimizers.Adam(learning_rate={{choice([0.001, 0.01, 0.1])}}),
# batch_size={{choice([8, 16, 32, 64, 128, 256])}},
#epochs=20

### Execução do Hyperas
Execução das funções com os códigos básicos do Hyperas retornando o melhor modelo.

In [7]:
def try_hyperas():
    try:
        best_run, best_model, space = optim.minimize(model=create_model,
                                                     data=data,
                                                     algo=tpe.suggest,
                                                     max_evals=5,
                                                     trials=Trials(),
                                                     eval_space=True,
                                                     notebook_name='hyperas_optimization',
                                                     return_space=True)
        
        return best_run, best_model, space
    
    except Exception as e:
        print(e.message)
        print(e)
        
        return None, None, None 
        
best_run, best_model, space = try_hyperas()

>>> Imports:
#coding=utf-8

try:
    import os
except:
    pass

try:
    import itertools
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import tensorflow_addons as tfa
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from tensorflow.python.keras.preprocessing.sequence import pad_sequences
except:
    pass

try:
    from tensorflow.python.keras.models import Model, Sequential
except:
    pass

try:
    from tensorflow.python.keras.layers import Input, Embedding, LSTM, Lambda, Conv1D, Dense, Dropout, Activation, Bidirectional
except:
    pass

try:
    from tensorflow.python.keras import backend as k
except:
    pass

try:
    from tensorflow.python.keras.layers import Lambda, Reshape, dot
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choic

Epoch 1/20                                                                                                                                                                                
765/765 - 41s - loss: 0.3322 - accuracy: 0.5197 - val_loss: 0.3222 - val_accuracy: 0.5324                                                                                                 

Epoch 2/20                                                                                                                                                                                
765/765 - 37s - loss: 0.3135 - accuracy: 0.5554 - val_loss: 0.3307 - val_accuracy: 0.5402                                                                                                 

Epoch 3/20                                                                                                                                                                                
765/765 - 37s - loss: 0.3110 - accuracy: 0.5613 - val_loss: 0.3

Epoch 4/20                                                                                                                                                                                
96/96 - 85s - loss: 0.3543 - accuracy: 0.4984 - val_loss: 0.3538 - val_accuracy: 0.5018                                                                                                   

Epoch 5/20                                                                                                                                                                                
96/96 - 85s - loss: 0.3584 - accuracy: 0.5045 - val_loss: 0.3610 - val_accuracy: 0.5034                                                                                                   

Epoch 6/20                                                                                                                                                                                
96/96 - 87s - loss: 0.3616 - accuracy: 0.5014 - val_loss: 0.361

### Melhores resultados
Resultado dos melhores hiperparâmetros encontrados assim como o evaluate dos valores de loss e métricas do modelo treinado.

Como é apresentado no [link](https://stackoverflow.com/questions/44843581/what-is-the-difference-between-model-fit-an-model-evaluate-in-keras) diferença entre:

- *fit()*: is for training the model with the given inputs (and corresponding training labels);

- *evaluate()*: is for evaluating the already trained model using the validation (or test) data and the corresponding labels. Returns the loss value and metrics values for the model;

- *predict()*: is for the actual prediction. It generates output predictions for the input samples.

Logo o hyperas realiza a avaliação do modelo treinado e não a predição de dados dado como entrada e com resultados de saída, o qual no caso do meu trabalho seriam valores entre 0 e 1, onde quanto mais próximo de 1 mais similar são duas instâncias de textos indicando que possivelmente são de mesmo autores. Caso contrário são assimilares e de autores distintos.

In [8]:
if best_run == None:
    print("It was not possible to perform your best model hyperparameters!")
else:
    x_train, y_train, x_test, y_test = data()
    
    print("Evalutation of best performing model:")
    print(best_model.evaluate([x_test['left'], x_test['right']], y_test))
    
    print("Best performing model chosen hyperparameters:")
    print(best_run)
    
    # realizar aqui o código que guarda em um arquivo .CSV os melhores hyperparâmetros

Evalutation of best performing model:
107/107 [==============================] - 5s 43ms/step - loss: 0.3219 - accuracy: 0.5626
[0.3219170868396759, 0.5626470446586609]
Best performing model chosen hyperparameters:
{'Activation': 'selu', 'Dropout': 0.11245974166556161, 'Dropout_1': 0.3207527760045966, 'LSTM': 128, 'batch_size': 32, 'epochs': 20, 'kernel_initializer': <tensorflow.python.keras.initializers.initializers_v2.VarianceScaling object at 0x0000024506657710>, 'loss': <tensorflow.python.keras.losses.MeanSquaredError object at 0x00000245065FAC88>, 'optimizer': <tensorflow.python.keras.optimizer_v2.adamax.Adamax object at 0x00000244FE6C4F28>}
